# Main Process
Stochastic Model + Nonlinear Model + Resilience Assessment

In [1]:
# import modules
import numpy as np
import pandas as pd
import openseespy as ops
import pathlib
import pickle
import random
# module for SGMM

# module for NTHA
from BuildingObject import Building_object
from beam_component import Beam
from column_component import Column
from steel_material import SteelMaterial
from nonlinear_analysis import NonlinearAnalysis
from StochasticGroundMotionModeling import StochasticGroundMotionModeling
from loss_calculation import Data

In [2]:
# BASE INFORMATION
cwdFile = pathlib.Path.cwd()
buildingDataFile = cwdFile /'BuildingData'
geometryFile = buildingDataFile / 'Geometry.csv'
memberSizeFile = buildingDataFile / 'MemberSize.csv'
loadsFile = buildingDataFile / 'Loads.csv'

member_size = pd.read_csv(memberSizeFile)
gravity_loads = pd.read_csv(loadsFile)
directory = {}
directory = {'building data': buildingDataFile}
building = Building_object(directory, member_size, gravity_loads)

# beams
beamSizeFile = buildingDataFile / 'beamsectionsize.csv'
SectionDatabaseFile = cwdFile / 'AllSectionDatabase.csv'
SectionDatabase = pd.read_csv(SectionDatabaseFile)

steel = SteelMaterial(yield_stress=50, ultimate_stress=65, elastic_modulus=29000, 
                      Ry_value=1.1)  # Unit: ksi
# 创建包含梁信息的嵌套字典
beam_section_size = pd.read_csv(beamSizeFile)
beams = {}
length = int(building.geometry['X bay width'])
for _, row in beam_section_size.iterrows():
    level, bay = map(int, row[:2])
    bsection_size = {'size': row[2]}
    beams.setdefault(level, {})
    beams[level].setdefault(bay, {})
    beams[level][bay] = Beam(bsection_size['size'], length, steel,SectionDatabase)
    
# elastic demand
elasticDemandFile = cwdFile / 'elastic_demand.pkl'
with open(elasticDemandFile, 'rb') as f:
    elastic_demand = pickle.load(f)

# column
columnSizeFile = buildingDataFile / 'columnsectionsize.csv'
SectionDatabaseFile = cwdFile / 'AllSectionDatabase.csv'
SectionDatabase = pd.read_csv(SectionDatabaseFile)

steel = SteelMaterial(yield_stress=50, ultimate_stress=65, elastic_modulus=29000, 
                      Ry_value=1.1)  # Unit: ksi

# 构建包含柱信息的嵌套字典
column_section_size = pd.read_csv(columnSizeFile)
columns = {}
for _, row in column_section_size.iterrows():
    story, pier = map(int, row[:2])
    csection_size = {'size': row[2]}
    columns.setdefault(story, {})
    columns[story].setdefault(bay, {})
    axial_demand = abs(elastic_demand.dominate_load['column axial'][story, 2 * pier])
    Lx = (building.geometry['floor height'][story+1] - building.geometry['floor height'][story]).item()
    Ly = Lx
    columns[story][pier] = Column(csection_size['size'], axial_demand, Lx, Ly, steel, SectionDatabase)

baseFile = cwdFile

In [3]:
nSample = 1
edpOutput = np.zeros((nSample, 8))
costOutput = np.zeros(nSample)
np.random.seed(2)  # 确保结果可以复现
for i in range(nSample):
    print(i)
    # whichone = random.randint(0, 999)
    ACC, tn = StochasticGroundMotionModeling(7.62, 19.3, 602, 0)
    # NTHA
    dt = 0.01
    ACC = ACC.tolist()
    edpResult = NonlinearAnalysis(building, columns, beams, baseFile, ACC, dt)
    edpOutput[i, :] = edpResult
    print(edpResult)
    data = Data()
    IDR = edpResult[:3]
    PFA = edpResult[3:7]
    frameCost, sframeCost, nframeCost, cframeCost = data.cal_repair(IDR, PFA)
    print(frameCost)
    costOutput[i] = frameCost

0


Modified Ibarra-Medina-Krawinkler Model with Bilinear Hysteretic Response


Analysis Completed!
[0.00487869 0.00476931 0.00265371 0.28023853 0.19000926 0.19075288
 0.22818147 0.0010101 ]
72814.20669990484


In [4]:
# import pickle
# import os
# os.chdir(cwdFile)
# file = open('edpOutput.pickle', 'wb')
# pickle.dump(edpOutput, file)
# file.close()

# file = open('costOutput.pickle', 'wb')
# pickle.dump(costOutput, file)
# file.close()

In [5]:
# with open('edpOutput.pickle', 'rb') as file:
#     dict_get = pickle.load(file)
# print(dict_get)

In [6]:
# import matplotlib.pyplot as plt
# plt.hist(costOutput, bins=50)

In [7]:
# IDR = np.array([4.89864183e-03, 4.80458308e-03, 2.76950154e-03])
# PFA = np.array([3.77184274e-01, 1.96116983e-01, 1.82918742e-01, 2.32016048e-01])
# data = Data()
# np.random.seed(1)  # 确保结果可以复现
# frameCost, sframeCost, nframeCost, cframeCost = data.cal_repair(IDR, PFA)
# frameCost

In [8]:
# # SGMM
# from StochasticGroundMotionModeling import StochasticGroundMotionModeling
# import matplotlib.pyplot as plt
# # StochasticGroundMotionModeling(M, R, Vs, whichone, num=100, tn=40, F=1)
# ACC, tn = StochasticGroundMotionModeling(6.61, 19.3, 602, 0)
# t = np.arange(0, tn + 0.01, 0.01)
# plt.plot(t, ACC, linewidth=0.5)
# plt.xlabel('time (s)')
# plt.ylabel('acc (g)')
# plt.title('acc history')
# plt.grid(True)
# plt.show()

In [9]:
# # NTHA
# dt = 0.01
# ACC = ACC.tolist()
# edpResult = NonlinearAnalysis(building, columns, beams, baseFile, ACC, dt)

In [10]:
# print(edpResult)

In [11]:
# edpResult[3:7]

In [12]:
# from loss_calculation import Data
# data = Data()
# IDR = edpResult[:3]
# PFA = edpResult[3:7]
# frameCost, sframeCost, nframeCost, cframeCost = data.cal_repair(IDR, PFA)

In [13]:
# frameCost

In [14]:
# import matplotlib.pyplot as plt
# import numpy as np
# t = np.arange(0, 4001*0.01+0.002, 0.001)

# print(t.shape)
# print(U.shape)
# plt.figure(figsize = (12,4))
# plt.plot(t, U, linewidth = 1)